In [1]:
import torch
from recbole.utils.case_study import full_sort_topk, full_sort_scores
from recbole.quick_start import load_data_and_model

/home/adhi/epm-recsys-dev/.env-epm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-20 11:26:40,120	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-12-20 11:26:40,262	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file="saved/[256, 128, 64, 32]_0001.pth",
)

20 Dec 11:26    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/epm
checkpoint_dir = saved/
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = True

Training Hyper Parameters:
epochs = 600
train_batch_size = 4096
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field

## Just for checking userID/ShiptoID

In [3]:
import pandas as pd

df = pd.read_parquet("raw_data/RatingMatrix.parquet")
df.head()

,ship_to_id,item_code,final_rating
0,EPM_34952,TCTLB,2.9
1,EPM_35444,TCTLB,2.7
2,EPM_3238950,TCTLB,1.9
3,EPM_136080,TCTLB,4.4
4,EPM_4393798,TCTLB,2.7


## Loop Here for each user

In [4]:
# uid_series = np.array([1, 2])  # internal user id series
# or you can use dataset.token2id to transfer external user token to internal user id
uid_series = dataset.token2id(dataset.uid_field, ["EPM_34952"])
uid_series

array([1])

In [5]:
topk_score, topk_iid_list = full_sort_topk(
    uid_series, model, test_data, k=10, device=config["device"]
)

In [6]:
print(topk_score)  # scores of top 10 items
# print(topk_iid_list)  # internal id of top 10 items
external_item_list = dataset.id2token(dataset.iid_field, topk_iid_list.cpu())
print(external_item_list)  # external tokens of top 10 items
print()



# score = full_sort_scores(uid_series, model, test_data, device=config["device"])
# print(score)  # score of all items
# print(
#     score[0, dataset.token2id(dataset.iid_field, ["10137"])]
# )  # score of item ['242', '302'] for user '196'.


tensor([[0.9886, 0.9707, 0.9690, 0.9628, 0.9519, 0.9459, 0.9436, 0.9358, 0.9339,
         0.9316]], device='cuda:0')
[['TSVND' 'KLSPB' 'LDPDA' 'TCETA' 'TSKAA' 'TGMPJ' 'MYGPB' 'KFNBD' 'BBF19'
  'TCDRA']]

